In [1]:
import pandas as pd

# auto 계정을 추려내기 위해서 다음의 데이터를 사용한다.
* 중복거래 데이터
* 활동데이터

In [2]:
tr = pd.read_csv('../3.통합/4.주당평균_파생변수/train_create_mean.csv')
te = pd.read_csv('../3.통합/4.주당평균_파생변수/test_create_mean.csv')
to = pd.concat([tr, te], sort=False).reset_index(drop=True)
source = pd.read_csv('auto_source_dup.csv')
target = pd.read_csv('auto_target_dup.csv')
hard = pd.read_csv('hard_list.csv')

In [3]:
to_source = pd.merge(to, source, on='acc_id', how='inner')
to_target = pd.merge(to, target, on='acc_id', how='inner')
to_hard = pd.merge(to, hard, on='acc_id', how='inner')

In [4]:
print(list(to.columns))

['acc_id', 'label', 'activated_wk', 'avg_cnt_dt', 'avg_play_time', 'avg_npc_exp', 'avg_npc_hongmun', 'avg_quest_exp', 'avg_quest_hongmun', 'avg_item_hongmun', 'avg_game_combat_time', 'avg_get_money', 'avg_duel_cnt', 'avg_duel_win', 'avg_partybattle_cnt', 'avg_partybattle_win', 'avg_cnt_enter_inzone_solo', 'avg_cnt_enter_inzone_light', 'avg_cnt_enter_inzone_skilled', 'avg_cnt_enter_inzone_normal', 'avg_cnt_enter_raid', 'avg_cnt_enter_raid_light', 'avg_cnt_enter_bam', 'avg_cnt_clear_inzone_solo', 'avg_cnt_clear_inzone_light', 'avg_cnt_clear_inzone_skilled', 'avg_cnt_clear_inzone_normal', 'avg_cnt_clear_raid', 'avg_cnt_clear_raid_light', 'avg_cnt_clear_bam', 'avg_normal_chat', 'avg_whisper_chat', 'avg_district_chat', 'avg_party_chat', 'avg_guild_chat', 'avg_faction_chat', 'avg_cnt_use_buffitem', 'avg_gathering_cnt', 'avg_making_cnt', 'avg_party_time', 'avg_party_count', 'avg_party_mem_count', 'avg_mean_party_time', 'avg_mean_party_mem', 'total_guild_mem', 'guild_count', 'max_guild_mem', '

In [5]:
# and 조건
sou_res = to_source[
    # 개미 계정은 레이드 참가가 불가능하다.
    (to_source['avg_cnt_enter_raid']==0)
    & (to_source['avg_cnt_enter_raid_light']==0)
    & (to_source['avg_cnt_enter_bam']==0)
    # 개미 계정은 캐쉬 충전을 하지 않는다.
    & (to_source['avg_payment_amount']==0)
    # 개미 계정은 돈과 잡화만 관리자에게 공급한다.
    & (to_source['avg_give_money_count']>0)
    & (to_source['avg_give_grocery_count']>0)
    & (to_source['avg_give_weapon_count']==0)
    & (to_source['avg_give_accessory_count']==0)
    & (to_source['avg_give_gem_count']==0)
    & (to_source['avg_give_costume_count']==0)
    # 개미 계정은 전투를 통해 재화를 번다.
    & (to_source['avg_game_combat_time']>=30)
    # 개미 계정은 길드가 없음
    & (to_source['guild_count']==0)
    # 동일 관리자 계정에 반복하여 재화를 공급한다.
    & (to_source['dup_give_count']>0)
    & (to_source['avg_give_money_sum']>to_source['avg_rec_money_sum'])
    & (to_source['avg_give_grocery_sum']>to_source['avg_rec_grocery_sum'])
]

# or 조건
sou_res = sou_res[
    (sou_res['avg_cnt_enter_inzone_solo']>0)
    | (sou_res['avg_cnt_enter_inzone_light']>0)
    | (sou_res['avg_cnt_enter_inzone_skilled']>0)
    | (sou_res['avg_cnt_enter_inzone_normal']>0)
].reset_index(drop=True)

In [6]:
# auto 기준과 hard 기준의 중복 계정
# 해당 계정들은 hard 유저가 아닌 auto 계정으로 판별됨.
auto_hard = pd.merge(sou_res, hard, on='acc_id', how='inner')
auto = auto_hard[auto_hard['avg_trade_count']>5]
hard = pd.DataFrame(set(hard['acc_id']) - set(auto['acc_id'])).rename(columns={0: 'acc_id'})
auto = pd.DataFrame(set(sou_res['acc_id']) - set(hard['acc_id'])).rename(columns={0: 'acc_id'})

In [7]:
auto['ant_id'] = 1
auto['master_id'] = 0
auto.head()

,acc_id,ant_id,master_id
0,bd2246b643465f2f920c14cf8fa52b53d119b3fc80d654...,1,0
1,1120f5119d9d0251df7a7a50389fbb297b4a8f2b96ff97...,1,0
2,b60d9d6277bffdd4d22c3ce2d8e68183be70e670dbef27...,1,0
3,8d485357d6e0a8b197d91c5259528bb188b1a5f59f6cb1...,1,0
4,c452fbfeb1e6360c42fb00f781ddde91a416c3e3f42b7d...,1,0


In [8]:
tar_res = to_target[
    (to_target['avg_cnt_enter_raid']==0)
    & (to_target['avg_cnt_enter_raid_light']==0)
    & (to_target['avg_payment_amount']==0)
    & (to_target['avg_rec_count']>10)
    & (to_target['avg_rec_money_sum']>1000)
    & (to_target['avg_rec_grocery_count']>0)
    & (to_target['dup_rec_acc_count']>1)
]

In [9]:
master = pd.DataFrame(tar_res['acc_id']).reset_index(drop=True).rename(columns={0: 'acc_id'})
hard = pd.DataFrame(set(hard['acc_id']) - set(master['acc_id'])).rename(columns={0: 'acc_id'})
master['ant_id'] = 0
master['master_id'] = 1
master.head()

,acc_id,ant_id,master_id
0,0b19aa719ec53186bc2b55e8809e71dcfa15eb6314e8e6...,0,1
1,0cf7cff311ce5f7de395711f7cba8667700e0e537fcefb...,0,1
2,0fa9635d0fbd779cdfe016de4544ca2f686533d2d71d25...,0,1
3,1453bef5863d9dc48510ffccf742f8d52249ea55605e91...,0,1
4,14555f035fb0af110fd483b8e0c1faa955d7fbfe716a0d...,0,1


In [10]:
auto_list = pd.concat([auto, master]).sort_values(by='acc_id').reset_index(drop=True)
print(auto_list.groupby(['ant_id', 'master_id']).count())
auto_list['auto_id']=1

                  acc_id
ant_id master_id        
0      1             120
1      0           21748


In [11]:
auto_list.to_csv('auto_list.csv', index=False)
hard.to_csv('hard_list_remove_auto.csv', index=False)